# Preprocesamiento de datos

En este notebook se abordarán las tareas de preprocesado y limpieza de datos para ser tratados más adelante en nuestro análisis exploratorio (EDA)

In [4]:
import pandas as pd

In [5]:
videogames_data = pd.read_csv('data/videogames_analysis.csv')

Antes que nada, eliminamos todas las columnas con un 100% de valores nulos o con un 100% de valores a 0 y el resto de columnas que no vamos a utilizar, ya que no van a tener ninguna utilidad en nuestro análisis.

In [6]:
videogames_data = videogames_data.drop(columns=['score_rank', 'genre', 'ccu', 'tags', 'userscore', 'release_year', 'discount', 'average_2weeks', 'median_2weeks'])

In [7]:
videogames_data.describe().T

,count,mean,std,min,25%,50%,75%,max
appid,1012.0,502304.610672,458212.805553,10.0,212725.00,342955.00,700537.50,2399830.00
positive,1012.0,78075.082016,260517.726373,0.0,12631.25,28121.50,66118.25,6892618.00
negative,1012.0,11751.269763,49211.487643,0.0,1754.50,3578.00,8211.00,982541.00
average_forever,1012.0,2031.487154,3895.124631,0.0,387.25,874.50,2045.75,55912.00
median_forever,1012.0,647.541502,1162.816669,0.0,137.00,296.50,688.25,15695.00
price,1012.0,14.618607,14.731289,0.0,0.00,12.08,18.59,65.09
initialprice,1012.0,16.753696,15.774100,0.0,0.00,13.94,27.89,65.09
languages,1012.0,7243.724308,51630.182967,0.0,73.50,410.50,1936.50,1086164.00


Separamos la columna owners en 4 para una mejor lectura y la eliminamos

In [8]:
# Creo nuevas columnas a partir de 'owners' dividiendo los valores
owners_split = videogames_data['owners'].str.split(' .. ', expand=True)
videogames_data['owners_min'] = pd.to_numeric(owners_split[0].str.replace(',', ''))
videogames_data['owners_max'] = pd.to_numeric(owners_split[1].str.replace(',', ''))

# Creo columnas de texto para propietarios mínimos y máximos
videogames_data['owners_min_text'] = owners_split[0]
videogames_data['owners_max_text'] = owners_split[1]



Formateamos columnas owners_min_text y owners_max_text

In [6]:
def format(value):
    num_value = float(value.replace(',', ''))
    return f"{num_value/1e6:.0f}M"

# Aplicar la función modificada a las columnas
videogames_data['owners_min_text'] = videogames_data['owners_min_text'].apply(format)
videogames_data['owners_max_text'] = videogames_data['owners_max_text'].apply(format)


In [7]:
videogames_data.head()

,appid,name,developer,publisher,positive,negative,owners,average_forever,median_forever,price,initialprice,languages,release_date,owners_min,owners_max,owners_min_text,owners_max_text
0,570,Dota 2,Valve,Valve,1777245,389764,"200,000,000 .. 500,000,000",39211,822,0.00,0.00,573082,2013-07-09,200000000,500000000,200M,500M
1,730,Counter-Strike: Global Offensive,Valve,Valve,6892618,982541,"100,000,000 .. 200,000,000",29752,5360,0.00,0.00,1086164,2012-08-21,100000000,200000000,100M,200M
2,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",1333940,965634,"50,000,000 .. 100,000,000",24257,6344,0.00,0.00,490082,2017-12-21,50000000,100000000,50M,100M
3,1063730,New World,Amazon Games,Amazon Games,191896,80619,"50,000,000 .. 100,000,000",10660,3893,37.19,37.19,12707,2021-09-28,50000000,100000000,50M,100M
4,440,Team Fortress 2,Valve,Valve,964115,62958,"50,000,000 .. 100,000,000",7732,319,0.00,0.00,85168,2007-10-10,50000000,100000000,50M,100M


Comprobamos el resto de valores nulos y los tratamos

In [8]:
null_values = videogames_data.isnull().sum()
print(null_values.sum())
print(null_values[null_values>0])

47
developer        1
publisher        2
release_date    44
dtype: int64


In [9]:
# Filtrar y mostrar filas donde la columna 'developer' es nula
null_developers = videogames_data[videogames_data['developer'].isnull()]
print("Filas donde 'developer' es nulo:")
print(null_developers)

# Filtrar y mostrar filas donde la columna 'publisher' es nula
null_publishers = videogames_data[videogames_data['publisher'].isnull()]
print("\nFilas donde 'publisher' es nulo:")
print(null_publishers)


Filas donde 'developer' es nulo:
      appid                              name developer publisher  positive  \
949  247120  Portal 2 Sixense Perceptual Pack       NaN   Sixense       295   

     negative                  owners  average_forever  median_forever  price  \
949       235  1,000,000 .. 2,000,000               77               7    0.0   

     initialprice  languages release_date  owners_min  owners_max  \
949           0.0          2          NaN     1000000     2000000   

    owners_min_text owners_max_text  
949              1M              2M  

Filas donde 'publisher' es nulo:
     appid            name       developer publisher  positive  negative  \
490  40800  Super Meat Boy       Team Meat       NaN     31660      1795   
926  12900       AudioSurf  Dylan Fitterer       NaN     10302       448   

                     owners  average_forever  median_forever  price  \
490  2,000,000 .. 5,000,000              297              92  13.94   
926  1,000,000 .. 2,000,0

Verificamos si las filas con valores nulos en 'developer' tienen un valor no nulo en 'publisher' y viceversa

In [10]:
null_developer = videogames_data[videogames_data['developer'].isnull()]
null_publisher = videogames_data[videogames_data['publisher'].isnull()]

In [11]:
null_developer

,appid,name,developer,publisher,positive,negative,owners,average_forever,median_forever,price,initialprice,languages,release_date,owners_min,owners_max,owners_min_text,owners_max_text
949,247120,Portal 2 Sixense Perceptual Pack,NaN,Sixense,295,235,"1,000,000 .. 2,000,000",77,7,0.0,0.0,2,NaN,1000000,2000000,1M,2M


In [12]:
null_publisher

,appid,name,developer,publisher,positive,negative,owners,average_forever,median_forever,price,initialprice,languages,release_date,owners_min,owners_max,owners_min_text,owners_max_text
490,40800,Super Meat Boy,Team Meat,NaN,31660,1795,"2,000,000 .. 5,000,000",297,92,13.94,13.94,92,2010-11-30,2000000,5000000,2M,5M
926,12900,AudioSurf,Dylan Fitterer,NaN,10302,448,"1,000,000 .. 2,000,000",543,297,9.29,9.29,11,2008-02-15,1000000,2000000,1M,2M


In [13]:
# Imputamos los valores nulos de 'developer' con los valores de 'publisher' y viceversa
videogames_data.loc[videogames_data['developer'].isnull(), 'developer'] = videogames_data['publisher']
videogames_data.loc[videogames_data['publisher'].isnull(), 'publisher'] = videogames_data['developer']


In [14]:
# Verificamos si la imputación ha tenido éxito
null_values = videogames_data.isnull().sum()
print(null_values.sum())
print(null_values[null_values>0])

44
release_date    44
dtype: int64


Observamos que ha tenido éxito, pero todavía tenemos 44 filas con valores nulos en 'Release date'. Como son pocos, hacemos una búsqueda por internet para cada videojuego y rellenamos las fechas. Para ello mostramos los nombres de los juegos con fechas de lanzamiento nulas.

In [15]:
games_with_null_release_dates = videogames_data[videogames_data['release_date'].isnull()]

# Mostrar los nombres de los juegos con fechas de lanzamiento nulas
print(games_with_null_release_dates[['name', 'release_date']])


                                                  name release_date
14                                            Palworld          NaN
26                                            Among Us          NaN
27               Grand Theft Auto IV: Complete Edition          NaN
60                                      Lethal Company          NaN
150                                    Human Fall Flat          NaN
164                     Assassin’s Creed IV Black Flag          NaN
173                     Middle-earth: Shadow of Mordor          NaN
175                                          Starfield          NaN
189           STAR WARS Battlefront II (Classic, 2005)          NaN
201                                            Blender          NaN
212                        Age of Empires II (Retired)          NaN
224              Call of Duty: Modern Warfare 3 (2011)          NaN
241                                    Forza Horizon 4          NaN
270                               BattleBit Rema

In [16]:
# Creamos un diccionario con los nombres de los videojuegos y las fechas de lanzamiento
release_dates = {
    "Palworld": "2024-01-19",
    "Among Us": "2021-12-14",
    "Grand Theft Auto IV: Complete Edition": "2010-10-29",
    "Lethal Company": "2023-10-23",
    "Human Fall Flat": "2016-07-22",
    "Assassin’s Creed IV Black Flag": "2013-10-19",
    "Middle-earth: Shadow of Mordor": "2014-09-30",
    "Starfield": "2023-08-29",
    "STAR WARS Battlefront II (Classic, 2005)": "2005-10-01",
    "Blender": "2002-10-13",
    "Age of Empires II (Retired)": "2013-04-01",
    "Call of Duty: Modern Warfare 3 (2011)": "2011-11-08",
    "Forza Horizon 4": "2018-10-02",
    "BattleBit Remastered": "2023-06-15",
    "Arma 2": "2009-06-26",
    "Party Animals": "2023-09-20",
    "The Outlast Trials": "2023-03-05",
    "The Day Before": "2023-12-07",
    "TrackMania Nations Forever": "2008-04-16",
    "PAYDAY 3": "2023-09-21",
    "Need For Speed: Hot Pursuit": "2010-11-16",
    "UNO": "2009-09-30",
    "ARMORED CORE VI FIRES OF RUBICON": "2023-08-25",
    "Source Filmmaker": "2012-06-27",
    "The Elder Scrolls IV: Oblivion Game of the Year Edition Deluxe": "2006-03-20",
    "Street Fighter 6": "2023-06-02",
    "Soundpad": "2016-10-09",
    "RuneScape": "2001-01-04",
    "Burnout Paradise: The Ultimate Box": "2009-02-05",
    "Mass Effect 2 (2010 Edition)": "2010-01-29",
    "Borderlands Game of the Year": "2019-04-03",
    "Risk of Rain (2013)": "2013-10-08",
    "Garbage Crew!": "2023-12-08",
    "Road 96": "2021-08-16",
    "REMNANT II": "2023-10-12",
    "Enshrouded": "2024-01-24",
    "ARK: Survival Ascended": "2023-10-25",
    "Love Is All Around": "1995-01-23",
    "Squad 44": "2018-08-09",
    "Project CARS 2": "2017-09-22",
    "Portal 2 Sixense Perceptual Pack": "2011-04-19",
    "SteamVR": "2011-04-19",
    "Woodle Tree Adventures": "2014-06-06",
    "Cities: Skylines II": "2023-10-24"
}

# Normalizamos los nombres y eliminamos espacios extra antes de la comparación
videogames_data['name'] = videogames_data['name'].str.strip().str.lower()

# Normalizamos las claves del diccionario de la misma manera
normalized_release_dates = {name.strip().lower(): date for name, date in release_dates.items()}

# Se actualiza la columna 'Release date' en el DataFrame usando los nombres normalizados
for name, date in normalized_release_dates.items():
    videogames_data.loc[videogames_data['name'] == name, 'release_date'] = date

videogames_data.head() 


,appid,name,developer,publisher,positive,negative,owners,average_forever,median_forever,price,initialprice,languages,release_date,owners_min,owners_max,owners_min_text,owners_max_text
0,570,dota 2,Valve,Valve,1777245,389764,"200,000,000 .. 500,000,000",39211,822,0.00,0.00,573082,2013-07-09,200000000,500000000,200M,500M
1,730,counter-strike: global offensive,Valve,Valve,6892618,982541,"100,000,000 .. 200,000,000",29752,5360,0.00,0.00,1086164,2012-08-21,100000000,200000000,100M,200M
2,578080,pubg: battlegrounds,"KRAFTON, Inc.","KRAFTON, Inc.",1333940,965634,"50,000,000 .. 100,000,000",24257,6344,0.00,0.00,490082,2017-12-21,50000000,100000000,50M,100M
3,1063730,new world,Amazon Games,Amazon Games,191896,80619,"50,000,000 .. 100,000,000",10660,3893,37.19,37.19,12707,2021-09-28,50000000,100000000,50M,100M
4,440,team fortress 2,Valve,Valve,964115,62958,"50,000,000 .. 100,000,000",7732,319,0.00,0.00,85168,2007-10-10,50000000,100000000,50M,100M


In [17]:
# Verificamos si la imputación ha tenido éxito
null_values = videogames_data.isnull().sum()
print(null_values.sum())
print(null_values[null_values>0])

0
Series([], dtype: int64)


In [18]:
videogames_data.to_csv('data/videogames_processed.csv', index=False)